In [ ]:
#@title ### 1. Configuration {display-mode: "form"}

#@markdown # 1. models
official = False  #@param {'type': 'boolean'}
anything = True #@param {'type': 'boolean'}
nai = False  #@param {'type': 'boolean'}
waifu = False  #@param {'type': 'boolean'}
counterfait = False  #@param {'type': 'boolean'}

#@markdown # 2. additional features
#@markdown #### (although xformers is highly recommended since it greatly accelerate picture generation, building it from source currently doesn't work)
xformers = "from_pypi" #@param ["from_source", "from_pypi", "none"]
additional_server_parameters = "--listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue" # @param {type:"string"}
disabled_extensions = [] #@param
google_drive_mode = "download" # @param ["download", "upload", "none"]

#@markdown # 3. debug options
#@markdown #### (if the code suddenly stops running and displays ^C while it is loading weights, tick the following box)
oHNo_My_SErVEr_keEpS_cRaShiNg_iTSeLF_HeLP_mE = False #@param {'type': 'boolean'}

In [ ]:
#@title ### 2. Installation

from IPython.display import clear_output
%cd /content/
!mkdir -p /content/webui/ /content/buildspace/ /content/tools/ /content/drive/


# mount google drive
if google_drive_mode != "none":
  from google.colab import drive
  drive.mount('/content/drive/')


## download tool & system updates
!echo "updating system......"
!apt update -y
!apt dist-upgrade -y
clear_output()


## webui
!echo "installing webui......"
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git /content/webui/
%cd /content/webui/
!git pull
%pip install -r requirements.txt


## extensions for webui
extensions = {
    "https://github.com/AUTOMATIC1111/stable-diffusion-webui-aesthetic-gradients.git": "aesthetic-gradients",
    "https://github.com/yfszzx/stable-diffusion-webui-images-browser.git": "images-browser",
    "https://github.com/deforum-art/deforum-for-automatic1111-webui.git": "deforum",
    "https://github.com/7eu7d7/DreamArtist-sd-webui-extension.git": "dreamartist",
    "https://github.com/ThereforeGames/unprompted.git": "unprompted",
    "https://github.com/animerl/novelai-2-local-prompt.git": "nai-prompt-converter",
    "https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN.git": "localization-zhcn",
    "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git": "tagcomplete"
}
for repo,name in extensions.items():
  if name not in disabled_extensions:
    !echo "installing extension {name}......"
    !git clone {repo} /content/webui/extensions/{name}/
    %cd /content/webui/extensions/{name}/
    !git pull


## xformers
if xformers == "from_source":
  !echo "installing xformers......"
  !git clone --recursive https://github.com/facebookresearch/xformers.git /content/buildspace/xformers/
  %cd /content/buildspace/xformers/
  %pip install -r requirements.txt
  %pip install -e .
elif xformers == "from_pypi":
  !echo "installing xformers......"
  %pip install torch torchvision torchaudio pytorch-lightning
  %pip install -U xformers
elif xformers == "none":
  pass


## models
!mkdir -p /content/webui/models/Stable-diffusion/ /content/webui/models/hypernetworks/ /content/webui/embeddings/
%cd /content/webui/models/Stable-diffusion/

if google_drive_mode != "download":
  # official stable diffusion
  if official == True:
    !echo "downloading official diffusion......"
    !curl -Lo official.ckpt https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt

  # anything
  if anything == True:
    !echo "downloading anything......"
    #!curl -Lo anything.ckpt https://huggingface.co/xyn-ai/anything-v4.0/resolve/main/anything-v4.5-pruned-fp32.ckpt # old version, will be using the newer one for now
    !curl -Lo anything.safetensors https://civitai.com/api/download/models/90854

  # novel ai
  if nai == True:
    !echo "downloading novel ai......"
    !curl -Lo nai.ckpt https://huggingface.co/acheong08/NovelAI/resolve/main/stableckpt/animefull-final-pruned/model.ckpt
    !curl -Lo nai.yaml https://huggingface.co/acheong08/NovelAI/resolve/main/stableckpt/animefull-final-pruned/config.yaml
    !curl -Lo nai.vae.pt https://huggingface.co/acheong08/NovelAI/resolve/main/stableckpt/animevae.pt

  # waifu diffusion
  if waifu == True:
    !echo "downloading waifu diffusion......"
    !curl -Lo waifu.ckpt https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-beta3-base-fp16.safetensors

  # counterfait
  if counterfait == True:
    !echo "downloading counterfait......"
    !curl -Lo counterfait.safetensors https://huggingface.co/gsdf/Counterfeit-V3.0/resolve/main/Counterfeit-V3.0_fp32.safetensors

  if google_drive_mode == "upload":
    !mkdir -p /content/drive/MyDrive/stable-diffusion-res/models/
    !rsync -a -h -r /content/webui/models/Stable-diffusion/ /content/drive/MyDrive/stable-diffusion-res/models/
else:
  !rsync -a -h -r /content/drive/MyDrive/stable-diffusion-res/models/ /content/webui/models/Stable-diffusion/


# embeddings
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive


## server tunneling tool
!echo "downloading server tunneling tool......"
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.5.0/bore-v0.5.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /content/tools/ && chmod +x /content/tools/bore
!curl -Lo /content/tools/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /content/tools/cloudflared

In [ ]:
#@title ### 3. Server

%cd /content/webui/
if oHNo_My_SErVEr_keEpS_cRaShiNg_iTSeLF_HeLP_mE == False:
  !python launch.py --medvram {additional_server_parameters} & /content/tools/bore local 7860 --to bore.pub & /content/tools/cloudflared tunnel --url localhost:7860
else:
  !python launch.py --lowvram {additional_server_parameters} & /content/tools/bore local 7860 --to bore.pub & /content/tools/cloudflared tunnel --url localhost:7860